In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, BatchNormalization, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (ModelCheckpoint, TensorBoard, ReduceLROnPlateau,
                                        CSVLogger, EarlyStopping)
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
df = pd.read_csv('train/train_features_emotion_labels.csv')

In [ ]:
X = pd.DataFrame(df[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       '11', '12', '13', '14', '15', '16', '17']])
y = pd.Series(df.label)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_test, y_train, y_test=train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [ ]:

# Build the model with Conv1D and BiLSTM layers
model = Sequential()

# 1st Conv1D layer (2048 filters, kernel size = 3, padding = 'same', with maxpooling)
model.add(Conv1D(2048, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
model.add(BatchNormalization())

# # 2nd Conv1D layer (1024 filters, kernel size = 3, padding = 'same', with maxpooling)
model.add(Conv1D(1024, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
model.add(BatchNormalization())

# 3rd Conv1D layer (512 filters, kernel size = 3, padding = 'same', with maxpooling)
model.add(Conv1D(512, kernel_size=3, strides=1,padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
model.add(BatchNormalization())

# Adding Bidirectional LSTM layer
model.add(Bidirectional(LSTM(256, return_sequences=True)))  # You can increase the number of units if required
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Bidirectional(LSTM(128)))  # You can increase the number of units if required
model.add(BatchNormalization())

# Dense layer with 64 units
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(8, activation='relu'))
model.add(Dropout(0.5))


# Final Dense layer with 6 units for output (emotions)
model.add(Dense(6, activation='softmax'))  # Output layer with 6 classes (emotions)

# Compile the model using Adam optimizer and sparse_categorical_crossentropy loss
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

 


callbacks_defn = [ModelCheckpoint(filepath = 'best_weights_emotion_audio.h5', monitor='val_accuracy',
                                       verbose=1, save_best_only=True,save_weights_only=False,mode='min'),
                                       EarlyStopping(monitor='val_loss',min_delta =0.01, patience=25, mode='min', restore_best_weights=True),
                                       ReduceLROnPlateau(monitor='val_loss', factor=0.75,patience=10, min_lr=0.000001)]

# Train the model with early stopping
history = model.fit(X_train, y_train, epochs=2000, batch_size=32, validation_data=(X_val, y_val), callbacks=callbacks_defn,
                       verbose = 1,
                       shuffle = True, )




In [ ]:

# Evaluate the model on the test data
y_pred = model.predict(X_test)

# Convert predictions from one-hot encoded to label format
y_pred_labels = np.argmax(y_pred, axis=1)  # Get the class with the highest probability
y_test_labels = y_test  # No need to convert y_test since we kept it as integers

# Evaluate the model performance
accuracy = accuracy_score(y_test_labels, y_pred_labels)
print(f"Accuracy on the test set: {accuracy * 100:.2f}%")

# Print the classification report
print("Classification Report:")
print(classification_report(y_test_labels, y_pred_labels))
